# Labs Access Analysis: 01 Data Download

# Setup

In [1]:
import pandas as pd
import numpy as np
import requests
from openrouteservice import client
import time
import json

In [2]:
# Load data
labs = pd.read_csv('data/labs.csv')

In [3]:
labs.head()

,Name,Type,City,Cluster,Address,CAP,Lat,Long,Source,URL,Notes
0,Fab Lab Barcelona,Fab Lab,Barcelona,BCN,"Carrer de Pujades, 102",8005.0,41.396925,2.194335,Fablabs.io,https://fablabbcn.org/,NaN
1,MADE Makerspace Barcelona,Fab Lab,Barcelona,BCN,Carrer Noguera Pallaresa 59-61,8014.0,41.373238,2.141084,"Fablabs.io, Hackerspaces.org, Makerspaces.make.co",NaN,NaN
2,Soko Tech,Fab Lab,Barcelona,BCN,"Carrer de Vallès i Ribot, 36",8027.0,41.424535,2.188883,Fablabs.io,https://soko.tech/,NaN
3,LSCongres,Fab Lab,Barcelona,BCN,"Carrer del Cardenal Tedeschini, 50",8027.0,41.426002,2.182026,Fablabs.io,https://fablabs.io/labs/lscongres,NaN
4,Green Fab Lab,Fab Lab,Cerdanyola del Vallès,BCN,"Ctra. BV-1415 (Horta-Cerdanyola), km 7",8290.0,41.450394,2.133441,Fablabs.io,NaN,NaN


In [4]:
closed_labs = labs[labs.Notes.str.contains("losed")==True]

In [5]:
valid_labs = labs[~labs.index.isin(closed_labs.index)]

In [6]:
valid_labs

,Name,Type,City,Cluster,Address,CAP,Lat,Long,Source,URL,Notes
0,Fab Lab Barcelona,Fab Lab,Barcelona,BCN,"Carrer de Pujades, 102",8005.0,41.396925,2.194335,Fablabs.io,https://fablabbcn.org/,NaN
1,MADE Makerspace Barcelona,Fab Lab,Barcelona,BCN,Carrer Noguera Pallaresa 59-61,8014.0,41.373238,2.141084,"Fablabs.io, Hackerspaces.org, Makerspaces.make.co",NaN,NaN
2,Soko Tech,Fab Lab,Barcelona,BCN,"Carrer de Vallès i Ribot, 36",8027.0,41.424535,2.188883,Fablabs.io,https://soko.tech/,NaN
3,LSCongres,Fab Lab,Barcelona,BCN,"Carrer del Cardenal Tedeschini, 50",8027.0,41.426002,2.182026,Fablabs.io,https://fablabs.io/labs/lscongres,NaN
4,Green Fab Lab,Fab Lab,Cerdanyola del Vallès,BCN,"Ctra. BV-1415 (Horta-Cerdanyola), km 7",8290.0,41.450394,2.133441,Fablabs.io,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
63,Canodrom,City Space,Barcelona,BCN,"Carrer de Concepción Arenal, 165",8027.0,41.427734,2.183339,Ajuntament de Barcelona policy document,https://canodrom.barcelona/,NaN
64,Cibernàrium Nou Barris,Incubator/Accelerator,Barcelona,BCN,"Carrer Marie Curie, 8-14",8042.0,41.437469,2.169056,Ajuntament de Barcelona policy document,https://cibernarium.barcelonactiva.cat/ciberna...,NaN
65,Cibernàrium 22@,Incubator/Accelerator,Barcelona,BCN,"Roc Boronat, 117",8018.0,41.403164,2.194483,Ajuntament de Barcelona policy document,https://cibernarium.barcelonactiva.cat/en/edif...,NaN
66,Fundació Comtal,Foundation,Barcelona,BCN,"Carrer Forn de la Fonda, 5",8003.0,41.386901,2.179529,Ajuntament de Barcelona policy document,https://comtal.org/,NaN


In [7]:
valid_labs.to_csv('data/valid_labs.csv')

In [8]:
valid_labs = valid_labs.astype({"Lat": float, "Long": float})

In [9]:
# 900 seconds = 15 minutes (city)
body = {"locations":[],"range":[900],"attributes":["total_pop","area","reachfactor"],"range_type":"time","interval":900,"area_units":"m","location_type":"start"}

locations = {}

In [10]:
for index, row in valid_labs.iterrows():
    if np.isnan(row["Lat"]) == False and np.isnan(row["Long"]) == False:
      locations[row["Name"]] = [row["Long"], row["Lat"]]

In [11]:
print(locations)

{'Fab Lab Barcelona': [2.194335155490849, 41.39692498100806], 'MADE Makerspace Barcelona': [2.141084282475449, 41.373238104403214], 'Soko Tech': [2.1888825978201867, 41.42453540945304], 'LSCongres': [2.1820259689844854, 41.426002192952815], 'Green Fab Lab': [2.133441411313727, 41.45039383574725], 'Barcelona Mobility LAB': [2.199793097820458, 41.43215238774583], 'Fab Lab Sant Cugat': [2.0917035978216068, 41.46756916570972], 'The FabLab: Make in Milano': [9.207912905011613, 45.4440458211788], 'Vectorealism FabLab': [9.231811340285022, 45.43830458381551], 'DamASpace': [9.1798175115845, 45.45245728345691], 'Miocugino - officina di gesta e ingegno': [9.16895578261383, 45.44821912386584], 'Opendot': [9.22096801664852, 45.45034823105788], 'Polifactory': [9.166295669123226, 45.506449418184594], 'WeMake | Makerspace Fablab': [9.1751237151523, 45.499039020857865], 'FabLab Ventura': [9.245129470972836, 45.4842083864999], 'FabLabMilano - Fare le Idee': [9.152947547090642, 45.50542134145349], 'Mult

In [17]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    # Get the authorization API key here: https://openrouteservice.org/
    'Authorization': '',
    'Content-Type': 'application/json; charset=utf-8'
}

In [13]:
body["locations"] = []
data = []
for k,i in enumerate(locations):
    print(i, locations[i])
    body["locations"].append(locations[i])
    # 15 minutes city, at walking distance
    call = requests.post('https://api.openrouteservice.org/v2/isochrones/foot-walking', json=body, headers=headers)
    body["locations"] = []
    print(call.status_code, call.reason)
    lab_dict = call.json()
    lab_dict["name"] = i
    data.append(lab_dict)
    print("---")
    time.sleep(30)

Fab Lab Barcelona [2.194335155490849, 41.39692498100806]
200 OK
---
MADE Makerspace Barcelona [2.141084282475449, 41.373238104403214]
200 OK
---
Soko Tech [2.1888825978201867, 41.42453540945304]
200 OK
---
LSCongres [2.1820259689844854, 41.426002192952815]
200 OK
---
Green Fab Lab [2.133441411313727, 41.45039383574725]
200 OK
---
Barcelona Mobility LAB [2.199793097820458, 41.43215238774583]
200 OK
---
Fab Lab Sant Cugat [2.0917035978216068, 41.46756916570972]
200 OK
---
The FabLab: Make in Milano [9.207912905011613, 45.4440458211788]
200 OK
---
Vectorealism FabLab [9.231811340285022, 45.43830458381551]
200 OK
---
DamASpace [9.1798175115845, 45.45245728345691]
200 OK
---
Miocugino - officina di gesta e ingegno [9.16895578261383, 45.44821912386584]
200 OK
---
Opendot [9.22096801664852, 45.45034823105788]
200 OK
---
Polifactory [9.166295669123226, 45.506449418184594]
200 OK
---
WeMake | Makerspace Fablab [9.1751237151523, 45.499039020857865]
200 OK
---
FabLab Ventura [9.245129470972836, 4

In [14]:
# Check the full data
data

[{'type': 'FeatureCollection',
  'bbox': [2.181814, 41.386679, 2.206919, 41.408053],
  'features': [{'type': 'Feature',
    'properties': {'group_index': 0,
     'value': 900.0,
     'center': [2.1943253081862886, 41.396932544923246],
     'area': 4414692.78,
     'reachfactor': 0.8994,
     'total_pop': 85512.0},
    'geometry': {'coordinates': [[[2.181814, 41.394814],
       [2.181818, 41.394634],
       [2.181969, 41.39116],
       [2.182512, 41.388045],
       [2.182594, 41.387901],
       [2.182729, 41.387688],
       [2.182745, 41.387677],
       [2.186288, 41.387211],
       [2.189545, 41.387251],
       [2.192458, 41.386679],
       [2.194172, 41.38668],
       [2.199413, 41.386923],
       [2.20173, 41.387252],
       [2.201769, 41.38728],
       [2.20611, 41.391197],
       [2.206369, 41.391261],
       [2.206478, 41.3914],
       [2.20658, 41.39185],
       [2.206919, 41.395455],
       [2.206873, 41.397327],
       [2.206578, 41.402674],
       [2.205493, 41.406096],
      

In [15]:
# Check the data, first lab
data[0]

{'type': 'FeatureCollection',
 'bbox': [2.181814, 41.386679, 2.206919, 41.408053],
 'features': [{'type': 'Feature',
   'properties': {'group_index': 0,
    'value': 900.0,
    'center': [2.1943253081862886, 41.396932544923246],
    'area': 4414692.78,
    'reachfactor': 0.8994,
    'total_pop': 85512.0},
   'geometry': {'coordinates': [[[2.181814, 41.394814],
      [2.181818, 41.394634],
      [2.181969, 41.39116],
      [2.182512, 41.388045],
      [2.182594, 41.387901],
      [2.182729, 41.387688],
      [2.182745, 41.387677],
      [2.186288, 41.387211],
      [2.189545, 41.387251],
      [2.192458, 41.386679],
      [2.194172, 41.38668],
      [2.199413, 41.386923],
      [2.20173, 41.387252],
      [2.201769, 41.38728],
      [2.20611, 41.391197],
      [2.206369, 41.391261],
      [2.206478, 41.3914],
      [2.20658, 41.39185],
      [2.206919, 41.395455],
      [2.206873, 41.397327],
      [2.206578, 41.402674],
      [2.205493, 41.406096],
      [2.205303, 41.406236],
      [2

In [16]:
with open("data/isochrones.json", 'w') as outfile:
    json.dump(data, outfile)